In [1]:
'''
Author: Dhananjay Shettigar (Roll No. 8702)
Program written for TEIT DMBI practical on classification algorithm (Decision Tree using entropy and information gain)
'''

import pandas as pd
import numpy as np
import math

import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
# Seaborn inbuilt dataset
df = sns.load_dataset('iris')
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [3]:
# Target column separation
target_col = 'species'
target = df[target_col]
columns = [col for col in df.columns]
columns.pop()
columns

['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

In [4]:
# Convert target column from string to unique integers
le = LabelEncoder()
target = le.fit_transform(target)
target
df[target_col] = pd.Series(target)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df, target, test_size = 0.2, random_state = 43)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(120, 5) (30, 5) (120,) (30,)


In [6]:
# Target entropy at start
unique_counts = np.bincount(target)
probabilities = unique_counts / len(target)

sum_prob = 0
for p in probabilities:
    sum_prob += p * math.log2(p)
root_entropy = - sum_prob
root_entropy


1.584962500721156

In [7]:
# Entropy = -(Summation of P(target_class) * log2(P(target_class) where target_class in 0 -> num_classes)

def calcEntropy(col):
    counts = np.bincount(col)
    probabilities = counts / len(col)
    entropy = 0
    for p in probabilities:
        if p > 0:
            entropy += p * math.log2(p)
    if entropy == 0:
        return entropy
    return -entropy

# Split a column into 2 at mean value

# Information Gain = Entropy(target) before splitting - ( num_rows in left branch/total_rows * entropy(target) in left branch rows ) - ( num_rows in right branch/total_rows * entropy(target) in right branch rows )

def calcInformationGain(df, col, cur_entropy):
    mid = df[col].mean()
    left = df[df[col] <= mid]
    right = df[df[col] > mid]
    left_prob = left.shape[0] / df.shape[0]
    right_prob = right.shape[0] / df.shape[0]
    left_entropy = calcEntropy(left['species'])
    right_entropy = calcEntropy(right['species'])
    return (cur_entropy - (left_prob * left_entropy + right_prob * right_entropy)), mid, left, right



In [8]:
MAX_NODES = 100
num_nodes = 1
dtree = []

# Recursively build tree by splitting dataframe into 2 on each call
def buildDecisionTree(df, parent_index, branch):
    global num_nodes
    entropy = calcEntropy(df[target_col])
    highest_gain_col = {'gain': 0, 'col': None, 'mid': None, 'left': None, 'right': None}
    for col in columns:
        gain, mid, left, right = calcInformationGain(df, col, entropy)
        if gain >= highest_gain_col['gain']:
            highest_gain_col['gain'] = gain
            highest_gain_col['col'] = col
            highest_gain_col['mid'] = mid
            highest_gain_col['left'] = left
            highest_gain_col['right'] = right
    
    node = {}
    node['left'] = None
    node['right'] = None
    node['col'] = highest_gain_col['col']
    node['mid'] = round(highest_gain_col['mid'], 2)
    node['class'] = df[target_col].value_counts().idxmax()
    dtree.append(node)
    num_nodes += 1

    if parent_index != -1:
        dtree[parent_index][branch] = len(dtree) - 1

    if highest_gain_col['gain'] >= entropy:
        return
    else:
        new_index = len(dtree) - 1
        buildDecisionTree(highest_gain_col['left'], new_index, 'left')
        try:
            temp = highest_gain_col['right'].shape[0]
            buildDecisionTree(highest_gain_col['right'], new_index, 'right')
        except:
            print("Error")
            pass
        

buildDecisionTree(X_train, -1, None)
dtree


[{'left': 1, 'right': 4, 'col': 'petal_length', 'mid': 3.84, 'class': 1},
 {'left': 2, 'right': 3, 'col': 'petal_length', 'mid': 1.75, 'class': 0},
 {'left': None, 'right': None, 'col': 'petal_width', 'mid': 0.25, 'class': 0},
 {'left': None, 'right': None, 'col': 'petal_width', 'mid': 0.89, 'class': 1},
 {'left': 5, 'right': 18, 'col': 'petal_width', 'mid': 1.73, 'class': 2},
 {'left': 6, 'right': 7, 'col': 'petal_length', 'mid': 4.49, 'class': 1},
 {'left': None, 'right': None, 'col': 'petal_width', 'mid': 1.28, 'class': 1},
 {'left': 8, 'right': 13, 'col': 'petal_length', 'mid': 4.75, 'class': 1},
 {'left': 9, 'right': 12, 'col': 'petal_length', 'mid': 4.59, 'class': 1},
 {'left': 10, 'right': 11, 'col': 'petal_width', 'mid': 1.51, 'class': 1},
 {'left': None, 'right': None, 'col': 'petal_width', 'mid': 1.46, 'class': 1},
 {'left': None, 'right': None, 'col': 'petal_width', 'mid': 1.65, 'class': 2},
 {'left': None, 'right': None, 'col': 'petal_width', 'mid': 1.41, 'class': 1},
 {'le

In [9]:
y_pred = []
for ind in X_test.index:
    index = 0
    while True:
        col = dtree[index]['col']
        mid = dtree[index]['mid']
        if X_test[col][ind] <= mid:
            if dtree[index]['left'] == None:
                y_pred.append(dtree[index]['class'])
                break
            else:
                index = dtree[index]['left']
        else:
            if dtree[index]['right'] == None:
                y_pred.append(dtree[index]['class'])
                break
            else:
                index = dtree[index]['right']
print(y_pred)


[0, 0, 2, 1, 2, 0, 2, 1, 1, 1, 0, 2, 2, 0, 1, 1, 0, 0, 2, 2, 0, 0, 0, 2, 2, 2, 0, 1, 0, 0]


In [10]:

print("Classification report - \n", classification_report(y_test,y_pred))


Classification report - 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      0.88      0.93         8
           2       0.90      1.00      0.95         9

    accuracy                           0.97        30
   macro avg       0.97      0.96      0.96        30
weighted avg       0.97      0.97      0.97        30



In [11]:
X_train

,sepal_length,sepal_width,petal_length,petal_width,species
96,5.7,2.9,4.2,1.3,1
19,5.1,3.8,1.5,0.3,0
93,5.0,2.3,3.3,1.0,1
98,5.1,2.5,3.0,1.1,1
108,6.7,2.5,5.8,1.8,2
...,...,...,...,...,...
58,6.6,2.9,4.6,1.3,1
21,5.1,3.7,1.5,0.4,0
49,5.0,3.3,1.4,0.2,0
64,5.6,2.9,3.6,1.3,1
